In [1]:
#0. import von relevanten Bibliotheken
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

## Woche 4 - OBV und Bot Setup

In dieser Woche lernen wir den On-Balance Volume (OBV) Indikator kennen.

#### OBV
Definition:

$OBV_t = OBV_{t-1} + sign(\frac{p_t}{p_{t-1}}-1) * Volume_{t}$


#### Regression mit OBV
Wir nutzen den OBV als Input für unser Regression-Model. Anschließend erstellen wir eine Beispielabgabe (mit einem jobfile)

#### DecissionTree mit OBV
Wir nutzen den OBV als Input für unser DecissionTree-Model. Anschließend erstellen wir eine Beispielabgabe (mit stetigen readjustieren)

#### Statische SMA Abgabe
Wir zeigen auf, wie eine Abgabe zu Bot, der den SMA als Signalgeber nutzt aussehen kann


In [124]:
def OBV_berechnen(data:pd.DataFrame):
    #benötigt ein DataFrame mit den Spalten "4a. close (EUR)" und "5. volume"
    data["OBV"] = (np.sign(data["4a. close (EUR)"].pct_change()) * data["5. volume"]).cumsum()


In [125]:
def RSI_berechnen(data:pd.DataFrame,intervall:int):

  spalten_name = "RSI_"+str(intervall)

  # Bestimme die Preisänderung zum jeweiligen Zeitpunkt t-1
  delta = data["4a. close (EUR)"].diff()

  # Get rid of the first row, which has NaN values
  delta = delta[1:]

  # Calculate the gains and losses
  up = delta.where(delta > 0, 0)
  down = -delta.where(delta < 0, 0)

  # Calculate the rolling average of the gains and losses
  #window_size = 14 #als default
  avg_gain = up.rolling(intervall).mean()
  avg_loss = down.rolling(intervall).mean()

  # Calculate the relative strength
  rs = avg_gain / avg_loss

  # Calculate the RSI
  data[spalten_name] = 100 - (100 / (1 + rs))


In [ ]:
#1. laden von dem Datensatz
daten = pd.read_csv("btc_1h.csv",index_col="date")
daten = daten.sort_index(axis=0)
df1 = daten[["4a. close (EUR)","5. volume"]].copy()
df1.head()

In [353]:
RSI_berechnen(df1,17)

In [354]:
OBV_berechnen(df1)

In [ ]:
df1

In [356]:
df1["OBV_SMA"] = df1["OBV"].rolling(15).mean()
df1["OBV_SMA_DIF"] = df1["OBV"] - df1["OBV_SMA"]

In [357]:
#Prozentuale Änderung
df1["pct"] = df1["4a. close (EUR)"].pct_change().shift(-1)

In [ ]:
df1.tail(15)

In [359]:
df1 = df1.dropna()

In [ ]:
plt.scatter(df1["RSI_17"],df1["pct"])
plt.axhline(color="r")

In [ ]:
plt.scatter(df1["OBV_SMA_DIF"],df1["pct"])
plt.axhline(color="r")

In [ ]:
df1[["OBV","OBV_SMA"]].plot()

In [ ]:
df1[["OBV_SMA_DIF"]].plot()

In [364]:
df = df1.copy()

### Regression Beispiel

In [ ]:
#Linear Regession

#Training und Test Size aufteilen
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

model = LinearRegression()
model.fit(train_data[["OBV_SMA_DIF"]], train_data["pct"])

In [ ]:
nst = model.intercept_/((-1)*model.coef_)
nst

In [ ]:
#Was unsere Regression macht:
X_train = train_data["OBV_SMA_DIF"].values[:,np.newaxis]
y_train = train_data["pct"].values

plt.scatter(X_train,y_train,color="g")
plt.plot(X_train,model.predict(X_train),color="black")
plt.axhline(color="r")

nst = model.intercept_/((-1)*model.coef_)
nst

In [368]:
#Was unsere Regression macht:
X_test = test_data["OBV_SMA_DIF"].values[:,np.newaxis]
y_test = test_data["pct"].values
plt.scatter(X_test,y_test,color="g")
plt.plot(X_test,model.predict(X_test),color="black")
plt.axhline(color="r")

In [370]:
#predict
y1 = model.predict(df[["OBV_SMA_DIF"]].iloc[train_size:]) 

In [ ]:
#PF bestimmen
PF = pd.DataFrame(df["pct"].iloc[train_size:])
PF["pred"] = y1

PF["PF_17"] = (PF["pred"]>0.0) * PF["pct"]
PF["myPF_17"] = (PF["PF_17"] +1).cumprod()

PF["PF_BM"] =  PF["pct"]
PF["myPF_BM"] = (PF["PF_BM"] +1).cumprod()

PF.tail()

In [ ]:
#Plotte dein Ergebnis
PF[["myPF_BM","myPF_17"]].plot(figsize=(16,6))

### Baue einen Bot mit dem Regressionsmodel

mit joblib-File (somit "konstanter Trainingsdatensatz")

In [386]:
#1. speichere dein Model als pickle File ab
from joblib import dump, load
dump(model, 'regression1.joblib') 

In [455]:
def OBV_berechnen(data:pd.DataFrame):
    #benötigt ein DataFrame mit den Spalten "4a. close (EUR)" und "5. volume"
    data["OBV"] = (np.sign(data["4a. close (EUR)"].pct_change()) * data["5. volume"]).cumsum()

def Bot1(data:pd.DataFrame):
    #wir nehmen die letzten 1000 stündlichen Daten entgegen und verarbeiten diese
    #mit unserem Regressionsmodel aus dem joblib-File, um das Signal BTC oder EUR zurückzugegeben


    #bearbeite die Daten, sodass wir "OBV_SMA_DIF" als Signalgeber haben
    OBV_berechnen(data)

    data["OBV_SMA"] = data["OBV"].rolling(15).mean()
    data["OBV_SMA_DIF"] = data["OBV"] - data["OBV_SMA"]
    
    #lade das Regressionsmodel
    bot1_model = load("regression1.joblib")
    
    sig = bot1_model.predict(data[["OBV_SMA_DIF"]].iloc[-1:])
    
    if sig >0.0:
        OUT = "BTC"
    else:
        OUT = "EUR"

    return OUT
    

In [ ]:
df["OBV_SMA_DIF"].tail(25)

In [ ]:
daten_bot= daten[["4a. close (EUR)","5. volume"]].copy()
Bot1(daten_bot)

### DecissionTree Beispiel

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier


# Aufteilen der Daten in Trainings- und Testdaten
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

X_train = train_data[['5. volume','RSI_17', #'OBV', 'OBV_SMA',
         'OBV_SMA_DIF']].to_numpy()      
Y_train = (train_data["pct"]>0.0).to_numpy()

X_test = test_data[['5. volume', 'RSI_17' , #'OBV', 'OBV_SMA',
         'OBV_SMA_DIF']].to_numpy()                            
Y_test = (test_data["pct"]>0.0).to_numpy()


# Fit regression model
regr_1 = DecisionTreeClassifier(max_depth=15,random_state=1234)
regr_2 = DecisionTreeClassifier(max_depth=30,random_state=1234)    
regr_1.fit(X_train, Y_train)
regr_2.fit(X_train, Y_train)

In [ ]:
#predict
y_1 = regr_1.predict(X_test) 
y_2 = regr_2.predict(X_test)

PF = pd.DataFrame(df["pct"].iloc[train_size:])
PF["pred"] = y_1
PF["pred2"] = y_2

PF["PF_17"] = (PF["pred"]>0.0) * PF["pct"]
PF["myPF_17"] = (PF["PF_17"] +1).cumprod()

PF["PF_17v2"] = (PF["pred2"]>0.0) * PF["pct"]
PF["myPF_17v2"] = (PF["PF_17v2"] +1).cumprod()

PF["PF_BM"] =  PF["pct"]
PF["myPF_BM"] = (PF["PF_BM"] +1).cumprod()

PF.tail()

In [ ]:
PF[["myPF_BM","myPF_17","myPF_17v2"]].plot(figsize=(16,6))      #Classifier

### Baue Bot mit Decission Tree

neues Training des Modell mit den Input Daten, sodass dieses fortlaufend readjustiert wird


In [488]:
from sklearn.tree import DecisionTreeClassifier

def OBV_berechnen(data:pd.DataFrame):
    #benötigt ein DataFrame mit den Spalten "4a. close (EUR)" und "5. volume"
    data["OBV"] = (np.sign(data["4a. close (EUR)"].pct_change()) * data["5. volume"]).cumsum()

def RSI_berechnen(data:pd.DataFrame,intervall:int):

  spalten_name = "RSI_"+str(intervall)

  # Bestimme die Preisänderung zum jeweiligen Zeitpunkt t-1
  delta = data["4a. close (EUR)"].diff()

  # Get rid of the first row, which has NaN values
  delta = delta[1:]

  # Calculate the gains and losses
  up = delta.where(delta > 0, 0)
  down = -delta.where(delta < 0, 0)

  # Calculate the rolling average of the gains and losses
  #window_size = 14 #als default
  avg_gain = up.rolling(intervall).mean()
  avg_loss = down.rolling(intervall).mean()

  # Calculate the relative strength
  rs = avg_gain / avg_loss

  # Calculate the RSI
  data[spalten_name] = 100 - (100 / (1 + rs))

def bot2(data:pd.DataFrame):
    #verwendet alle Zeitpunkte bis auf den letzten zum Training des Decission Tree und gibt 
    #anschließend eine Prediction für das aktuelle Zeitintervall ab.
    #Das Modell trainiert sich bei Eingabe eines neuen DataFrame erneut.


    #bestimme RSI_17
    RSI_berechnen(data,17)

    #OBV_SMA_DIF bestimmen SMA intervall = 15
    OBV_berechnen(data)
    data["OBV_SMA"] = data["OBV"].rolling(15).mean()
    data["OBV_SMA_DIF"] = data["OBV"] - data["OBV_SMA"]
    
    #WICHTIG für DecissionTree
    data = data.dropna()

    #pct bestimmen und shiften
    data["pct"] = data["4a. close (EUR)"].pct_change().shift(-1)
    
    #Traindata 999h zuvor
    train_data = data.iloc[:-1]
    
    X_train = train_data[['5. volume','RSI_17','OBV_SMA_DIF']].to_numpy()      
    Y_train = (train_data["pct"]>0.0).to_numpy()

    #Setze das Model auf
    bot2_model = DecisionTreeClassifier(max_depth=15,random_state=1234)

    bot2_model.fit(X_train,Y_train)
    
    #aktuellsten Zeitpunkt für Prediction verwenden
    sig = bot2_model.predict(data[['5. volume','RSI_17','OBV_SMA_DIF']].iloc[-1:])
    
    if sig >0.0:            #bei Classification ist ">0.0" wichtig! 
        OUT = "BTC"
    else:
        OUT = "EUR"

    return OUT
    

In [ ]:
bot2(daten_bot.iloc[950:].copy())

### SMA Beispiel

In [501]:
def sma(data: pd.DataFrame,intervall:int):
  #diese Funktion berechnet den SMA_intervall von dem Input data
  #der SMA wird immer von dem Column "4a. close (EUR)" berechnet
  #ACHTUNG: Stelle sicher, dass data immer diese Spalte besitzt
  spalten_name = "SMA_"+str(intervall)
  data[spalten_name] = data["4a. close (EUR)"].rolling(intervall).mean()

sma(df,10)
sma(df,15)

df2 = df.dropna()

In [ ]:
df2.head(2)

In [ ]:
#SMA10 und SMA15 Kombination (für 1h intervalle)
df["Sig_TODO"] = (df["4a. close (EUR)"] > df["SMA_10"]) & (df["4a. close (EUR)"] > df["SMA_15"])
df["PF_TODO"] = df["Sig_TODO"] * df["pct"]
df["myPF_TODO"] = (df["PF_TODO"] +1).cumprod()

#Benchmark
df["PF_BM"] =  df["pct"]
df["myPF_BM"] = (df["PF_BM"] +1).cumprod()

df[["myPF_BM","myPF_TODO"]].plot(figsize=(16,6))


### Baue Bot mit statischen Signalen

kein ML; Sehr schnelle Berechnung!

In [506]:
def sma(data: pd.DataFrame,intervall:int):
  #diese Funktion berechnet den SMA_intervall von dem Input data
  #der SMA wird immer von dem Column "4a. close (EUR)" berechnet
  #ACHTUNG: Stelle sicher, dass data immer diese Spalte besitzt
  spalten_name = "SMA_"+str(intervall)
  data[spalten_name] = data["4a. close (EUR)"].rolling(intervall).mean()

def bot3(data:pd.DataFrame):
  #bestimmt den SMA10 sowie SMA15 und leitet aus diesen das Signal her

  sma(data,10)
  sma(data,15)

  data["Sig_BOT"] = (data["4a. close (EUR)"] > data["SMA_10"]) & (data["4a. close (EUR)"] > data["SMA_15"])
  
  sig = data["Sig_BOT"][-1]

  print(sig)
  
  if sig >0.0:            #bei Classification ist ">0.0" wichtig! 
    OUT = "BTC"
  else:
    OUT = "EUR"

  return OUT

In [ ]:
bot3(daten_bot)